In [ ]:
import os
import time
from googletrans import Translator

translator = Translator()
input_folder = r"C:/Temp/en_reorder"  # 번역할 파일들이 있는 폴더 경로
output_folder = os.path.join(input_folder, "kr")  # 번역된 파일 저장 폴더 경로

# kr 폴더 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 폴더 내 파일 리스트 가져오기
def get_files_with_en(folder_path):
    return [f for f in os.listdir(folder_path) if '_en_' in f and f.endswith('.txt')]

# 번역 및 저장 함수
def translate_file(input_path, output_path):
    try:
        with open(input_path, 'r', encoding='utf-8') as f:
            read_lines = f.readlines()
    except UnicodeDecodeError:
        print(f"UTF-8로 읽는 데 문제가 있습니다. cp949로 시도합니다: {input_path}")
        with open(input_path, 'r', encoding='cp949') as f_cp949:
            read_lines = f_cp949.readlines()

    # 빈 줄 필터링
    non_empty_lines = [line for line in read_lines if line.strip()]

    translated_lines = []
    for line in non_empty_lines:
        success = False
        retries = 3
        while not success and retries > 0:
            try:
                result = translator.translate(line, dest='ko')
                if result and result.text:  # 응답이 유효한지 확인
                    translated_lines.append(result.text)
                else:
                    translated_lines.append(f"번역 실패: {line.strip()}")
                success = True
                time.sleep(1)  # 요청 간 지연 추가
            except Exception as e:
                retries -= 1
                print(f"번역 오류 발생, 재시도 {3 - retries}/3: {e}")
                time.sleep(2 ** (3 - retries))  # 지수적 백오프

        if not success:
            translated_lines.append(f"번역 실패: {line.strip()}")

    # 빈 줄 복원
    output_lines = []
    index = 0
    for line in read_lines:
        if line.strip():
            output_lines.append(translated_lines[index])
            index += 1
        else:
            output_lines.append("")

    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write("\n".join(output_lines))
        print(f"번역된 내용이 {output_path}에 저장되었습니다.")
    except Exception as e:
        print(f"파일 저장 중 오류가 발생했습니다: {e}")

# 메인 실행
def main():
    files = get_files_with_en(input_folder)
    for file_name in files:
        input_path = os.path.join(input_folder, file_name)
        output_file_name = file_name.replace('_en_', '_kr_')
        output_path = os.path.join(output_folder, output_file_name)

        print(f"파일 번역 중: {file_name} -> {output_file_name}")
        translate_file(input_path, output_path)

if __name__ == "__main__":
    main()
